# Toxic Text Segmentation

Start by importing all dependencies

In [1]:
import csv
import chars2vec 
import re
import numpy as np
from tensorflow.keras import datasets, layers, models, losses, callbacks, Model
from sklearn.model_selection import train_test_split
import statistics

In [2]:
def f1(predictions, gold):
    """
    Calculates F1 score(a.k.a. DICE)
    
    Args:
        predictions: a list of predicted offsets
        gold: a list of offsets serving as the ground truth
        
    Returns: 
        a float score between 0 and 1
    """
    if len(gold) == 0:
        return 1 if len(predictions) == 0 else 0
    if len(predictions) == 0:
        return 0
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

Define a function to extract text

In [3]:
def read_text_data(filename):
    """
    Reads a csv file to extract text.
    
    Args:
        filename: a string specifying the filename / path
        
    Returns:
        A list of text sentences
    """
    data = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        count = 0
        for row in reader:
            data.append(row['text'])
    csvfile.close()
    return data

Define a function to extract spans

In [4]:
def read_data_span(filename):
    """
    Reads a csv file to extract the toxic spans.
    
    Args:
        filename: a string specifying the filename / path
        
    Returns:
        data: a list of strings of the toxic chars, 
        will look like '[1,2,3]' so it'll have to be split
        
    """
    data = []
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        count = 0
        for row in reader:
            data.append(row['span'])
    csvfile.close()
    return data

Read the training data

In [5]:
texts = read_text_data('data/tsd_train_readable.csv')
spans = read_data_span('data/tsd_train_readable.csv')
texts.extend(read_text_data('data/tsd_trial_readable.csv'))
spans.extend(read_data_span('data/tsd_trial_readable.csv'))


processed_texts = []
processed_spans = []
print(f"Lengths equal: {len(texts)==len(spans)}" + "\n")

Lengths equal: True



Get the maximum comment size (in no. of chars)

In [6]:
max_size = 0
for i in range(0, len(texts)-1):
    if len(texts[i]) > max_size:
        max_size = len(texts[i])
print(f"max size of sentence (in chars): {max_size}")

max size of sentence (in chars): 1000


Preprocess the training data, after analysis the max sentence size is 1000 characters long, also removing empty strings and split the spans in to actual lists

In [7]:
c2v_model = chars2vec.load_model('eng_50')
word_limit = 1024
for i in range(0, len(texts)-1):
    to_use = True
    if len(texts[i]) > word_limit:
        to_use = False
    if texts[i] == "":
        to_use = False
    new_spans = [int(j) for j in spans[i][1:-1].split(", ")]
    if max(new_spans) > len(texts[i]) - 1:
        to_use = False
    if to_use:
        if spans[i] != []:
            full_span = [[0,0,1] for j in range(0, word_limit)]
            for char_offset in new_spans:
                full_span[char_offset] = [1,0,0]
            for j in range(0, len(texts[i])-1):
                if full_span[j][1] == 0 and full_span[j][2] == 1:
                    full_span[j] = [0,1,0]
        else:
            full_span = [[1,0,0] for j in range(0, len(texts[i]))]           
        processed_texts.append(texts[i])
        processed_spans.append(full_span)

Define the training and testing datasets with numpy zero arrays, this is to allow us to pad the end
Of the toxic span with zeros as it is a fully convolutional network

In [8]:
train_Y = np.zeros(shape=(len(processed_spans), 1024, 3))
train_X = np.zeros(shape=(len(processed_texts), 1024, 50))         

Terrible Python best practise but you might wanna manually free up some memory. This is going to be a very large compuation

In [9]:
del texts
del spans

Build Train_X

In [10]:
for x, string in enumerate(processed_texts):
    for y, char in enumerate(string):
            char_vect = c2v_model.vectorize_words([char])
            train_X[x][y] = [word_vect for word_vect in char_vect[0]]

Build train_Y

In [11]:
for x, label in enumerate(processed_spans):
    for y, output in enumerate(label):
        train_Y[x][y] = output

Split train_X and train_Y into training and validation datasets.

In [12]:
train_X, val_X, train_Y, val_Y = train_test_split(train_X, train_Y, test_size=0.1, random_state=42)

Confirm the shape of the train and val datasets, should be ([sample_size], 1000, 50) and ([sample_size], 1000, 3) respectively

In [13]:
print(train_X.shape)
print(train_Y.shape)

(7102, 1024, 50)
(7102, 1024, 3)


As a global variable would be out of scope for the callback object class manually create a HighScore class

In [14]:
class HighScore:
    def __init__(self):
        self.high_score = 0
    def get_high_score(self):
        return self.high_score
    def set_high_score(self, new_score):
        self.high_score = new_score
high_score = HighScore()

Free up more memory

In [15]:
del processed_texts
del processed_spans

A prediction callback to act as a validation step, as the tensor is of a different shape to the F1 score of SemEval we must Convert it into it's proper form before checking the F1 score.

In [28]:
class PredictionCallback(callbacks.Callback):    
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(val_X)
        scores = []
        for x, pred in enumerate(y_pred):
            score = f1([j for j, i in enumerate(pred) if np.argmax(i) == 0], [j for j, i in enumerate(val_Y[x]) if np.argmax(i) == 0])
            scores.append(score)
        score = statistics.mean(scores)
        if score > high_score.get_high_score():
            high_score.set_high_score(score)
            model.save(f"UNet_checkpoint_sko")
        print(f"F1 score: {score}")

In [35]:
def get_SegNet(is_deep=False, is_sko=False):
    
    if is_sko:
        final_output_layer = 1
    else:
        final_output_layer = 9
        
    model = models.Sequential()
    model.add(layers.Input(shape = train_X.shape[1:]))

    model.add(layers.Conv1D(filters=32, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.MaxPooling1D(strides=2))

    model.add(layers.Conv1D(filters=64, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.MaxPooling1D(strides=2))

    model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(filters=128, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.MaxPooling1D(strides=2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(filters=256, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.MaxPooling1D(strides=2))
    model.add(layers.Dropout(0.3))
    
    if is_deep:
        model.add(layers.Dropout(0.3))
        model.add(layers.Conv1D(filters=512, kernel_size=9, strides=1, padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.ELU())
        model.add(layers.MaxPooling1D(strides=2))
        model.add(layers.Dropout(0.3))

        model.add(layers.UpSampling1D(size=2))
        model.add(layers.Conv1D(filters=512, kernel_size=9, strides=1, padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.ELU())
        model.add(layers.Dropout(0.3))

    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1D(filters=256, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.Dropout(0.3))

    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1D(filters=128, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.Dropout(0.3))

    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1D(filters=64, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())

    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1D(filters=32, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())

    model.add(layers.Conv1D(filters=3, kernel_size=final_output_layer, strides=1, padding='same', activation='softmax'))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

def get_DeconvNet(is_deep=False, is_sko=False):
    
    if is_sko:
        final_output_layer = 1
    else:
        final_output_layer = 9
        
    model = models.Sequential()
    model.add(layers.Input(shape = train_X.shape[1:]))

    model.add(layers.Conv1D(filters=32, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.MaxPooling1D(strides=2))

    model.add(layers.Conv1D(filters=64, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.MaxPooling1D(strides=2))

    model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(filters=128, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.MaxPooling1D(strides=2))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(filters=256, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.MaxPooling1D(strides=2))
    model.add(layers.Dropout(0.3))
    
    if is_deep:
        model.add(layers.Dropout(0.3))
        model.add(layers.Conv1D(filters=512, kernel_size=9, strides=1, padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.ELU())
        model.add(layers.MaxPooling1D(strides=2))
        model.add(layers.Dropout(0.3))

        model.add(layers.UpSampling1D(size=2))
        model.add(layers.Conv1DTranspose(filters=512, kernel_size=9, strides=1, padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.ELU())
        model.add(layers.Dropout(0.3))

    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1DTranspose(filters=256, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.Dropout(0.3))

    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1DTranspose(filters=128, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())
    model.add(layers.Dropout(0.3))

    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1DTranspose(filters=64, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())

    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1DTranspose(filters=32, kernel_size=9, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ELU())

    model.add(layers.Conv1D(filters=3, kernel_size=final_output_layer, strides=1, padding='same', activation='softmax'))
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

def get_UNet(is_deep=False, is_sko=False):
    
    if is_sko:
        final_output_layer = 1
    else:
        final_output_layer = 9
        
    if is_deep:
        encoder = [64, 128, 256, 512]
        decoder = [512, 256, 128, 64, 32]
    else:
        encoder = [64, 128, 256]
        decoder = [256, 128, 64, 32]
        
    # Entry conv block
    inputs = layers.Input(shape = train_X.shape[1:])
    x = layers.Conv1D(32, 3, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ELU()(x)
    x = layers.MaxPooling1D(strides=2, padding="same")(x)
    
    previous_block_activation = x
    
    for filters in encoder:
        x = layers.Conv1D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.ELU()(x)
        x = layers.MaxPooling1D(strides=2, padding="same")(x)

        residual = layers.Conv1D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual


    for filters in decoder:
        x = layers.Conv1DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.ELU()(x)
        x = layers.UpSampling1D(size=2)(x)

        # Project residual
        residual = layers.UpSampling1D(size=2)(previous_block_activation)
        residual = layers.Conv1D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Classification layer
    outputs = layers.Conv1D(3, final_output_layer, activation="softmax", padding="same")(x)

    # Define the model
    model = Model(inputs, outputs)
    
    # Compile model
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    return model

Get the model and view it's structure.

In [36]:
model = get_UNet(is_deep=False, is_sko=False)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 1024, 50)]   0                                            
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 1024, 32)     4832        input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_17 (BatchNo (None, 1024, 32)     128         conv1d_22[0][0]                  
__________________________________________________________________________________________________
elu_16 (ELU)                    (None, 1024, 32)     0           batch_normalization_17[0][0]     
____________________________________________________________________________________________

Train the model

In [ ]:
history = model.fit(train_X, train_Y, epochs=300, batch_size=32, callbacks=[PredictionCallback()])

Epoch 1/300
222/222 [==============================] - 31s 92ms/step - loss: 0.3081 - accuracy: 0.9482
F1 score: 0
Epoch 2/300
222/222 [==============================] - 21s 93ms/step - loss: 0.0599 - accuracy: 0.9820
F1 score: 0.11845222617867487
Epoch 3/300
222/222 [==============================] - 20s 90ms/step - loss: 0.0568 - accuracy: 0.9827
F1 score: 0.18874367261018118
Epoch 4/300
222/222 [==============================] - 20s 92ms/step - loss: 0.0532 - accuracy: 0.9833
F1 score: 0.21298432632411257
Epoch 5/300
222/222 [==============================] - 20s 92ms/step - loss: 0.0515 - accuracy: 0.9838
INFO:tensorflow:Assets written to: UNet_checkpoint_sko\assets
F1 score: 0.2510789061411174
Epoch 6/300
222/222 [==============================] - 20s 92ms/step - loss: 0.0494 - accuracy: 0.98460s - loss: 0.0494 - ac
F1 score: 0.24648864400261503
Epoch 7/300
222/222 [==============================] - 20s 91ms/step - loss: 0.0492 - accuracy: 0.9843
INFO:tensorflow:Assets written to:

222/222 [==============================] - 20s 91ms/step - loss: 0.0027 - accuracy: 0.9990
F1 score: 0.4520137759636264
Epoch 57/300
222/222 [==============================] - 20s 89ms/step - loss: 0.0025 - accuracy: 0.9991
F1 score: 0.4547932154759902
Epoch 58/300
222/222 [==============================] - 20s 91ms/step - loss: 0.0028 - accuracy: 0.9990
F1 score: 0.40245312062657207
Epoch 59/300
222/222 [==============================] - 20s 90ms/step - loss: 0.0056 - accuracy: 0.9980
F1 score: 0.3724416850271983
Epoch 60/300
222/222 [==============================] - 20s 90ms/step - loss: 0.0106 - accuracy: 0.9960
F1 score: 0.3818988450198352
Epoch 61/300
222/222 [==============================] - 21s 93ms/step - loss: 0.0094 - accuracy: 0.9965
F1 score: 0.43768835175352966
Epoch 62/300
222/222 [==============================] - 20s 89ms/step - loss: 0.0040 - accuracy: 0.9986
F1 score: 0.43927287991996355
Epoch 63/300
222/222 [==============================] - 20s 89ms/step - loss: 0

222/222 [==============================] - 20s 89ms/step - loss: 0.0027 - accuracy: 0.9991
F1 score: 0.47139467354039716
Epoch 114/300
222/222 [==============================] - 20s 91ms/step - loss: 0.0012 - accuracy: 0.9996
F1 score: 0.46064540075091714
Epoch 115/300
222/222 [==============================] - 20s 91ms/step - loss: 7.2579e-04 - accuracy: 0.9998
F1 score: 0.42890063598371886
Epoch 116/300
222/222 [==============================] - 21s 93ms/step - loss: 8.0770e-04 - accuracy: 0.9997
F1 score: 0.4789163895270474
Epoch 117/300
222/222 [==============================] - 20s 91ms/step - loss: 9.2308e-04 - accuracy: 0.9997
F1 score: 0.46001576309377656
Epoch 118/300
222/222 [==============================] - 21s 92ms/step - loss: 0.0033 - accuracy: 0.9989
F1 score: 0.4423820446999964
Epoch 119/300
222/222 [==============================] - 20s 91ms/step - loss: 0.0060 - accuracy: 0.9979
F1 score: 0.45265168097951053
Epoch 120/300
222/222 [==============================] - 20

Save model and check final validation score

In [ ]:
model.save(f"UNet_model_300_epochs_sko")
scores = []
pred_Y = model.predict(val_X)

for x, pred in enumerate(pred_Y):
    y_pred_f1_compatible = [j for j, i in enumerate(pred) if np.argmax(i) == 0]
    y_true_f1_compatible = [j for j, i in enumerate(val_Y[x]) if np.argmax(i) == 0]
    score = f1(y_pred_f1_compatible, y_true_f1_compatible)
    scores.append(score)

print('avg F1 %g' % statistics.mean(scores))


Manually view some predictions to check validity

In [ ]:
for x, pred in enumerate(pred_Y):
    score = f1([j for j, i in enumerate(pred) if np.argmax(i) == 0], [j for j, i in enumerate(val_Y[x]) if np.argmax(i) == 0])
    print(f"F1 score: {score}")
    print(f"Predicted categorical: {[np.argmax(i) for i in pred]}")
    print(f"Predicted span: {[j for j, i in enumerate(pred) if np.argmax(i) == 0]}")
    print(f"Ground truth span: {[j for j, i in enumerate(val_Y[x]) if np.argmax(i) == 0]}" + "\n")
    if x == 100:
          break
    